In [19]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from netCDF4 import Dataset
import os
import tqdm
import datetime


hem='s'
mission='cs'
bbox=True


server_dir = '/home/robbie/uit_mnt'
satam_directory = f'{server_dir}/Data/romal7177/ResearchData/IFT/EarthObservation/SatelliteAltimetry/'
scratch=f'{server_dir}/scratch/robbie/nrcs/pickles'
nh_it_dir=f'{satam_directory}OSISAF Sea Ice Type'
sh_it_dir = f'{server_dir}/scratch/robbie/nrcs/ice_type'
sh_latf = 'south_lat_12km.hdf'
sh_latgrid = Dataset(f'{sh_it_dir}/{sh_latf}')
sh_latgrid = np.array(sh_latgrid['Latitude [degrees]'])

# make NH october masks

october_masks = {}

for year in np.arange(2010,2023):
    
    month_folder = f'{nh_it_dir}/{year}/10/'
    
    type_string = f'{year}{10}01'
    
    f = f'ice_type_nh_polstere-100_multi_{type_string}1200.nc'

    d_type = Dataset(f'{month_folder}{f}')
    
    longrid = np.array(d_type['lon'])
    latgrid = np.array(d_type['lat'])
    
    ice_type = np.array(d_type['ice_type'][0])
    ice_type[ice_type<2] = 0
    ice_type[ice_type>4] = 0
    ice_type[ice_type>0] = 1
    ice_type[latgrid>87] = 1
    
    october_masks[year] = ice_type

nh_longrid = np.array(d_type['lon'])
nh_latgrid = np.array(d_type['lat'])

latgrids={'n':nh_latgrid,'s':sh_latgrid}

def get_ice_type_from_date(date,hem):
    
    if hem == 'n':
          
        f = f'{date.year}/{str(date.month).zfill(2)}/ice_type_nh_polstere-100_multi_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}1200.nc'
        with Dataset(f'{nh_it_dir}/{f}') as d:

            it = np.array(d['ice_type'][0]).astype(np.float32)
            it[it!=2]=0
            it[it==2]=1
            fyi = it.copy()
            
            it = np.array(d['ice_type'][0]).astype(np.float32)
            
            it[it!=3]=0
            it[it==3]=1
            myi = it.copy()
            
 
            return {'fyi':fyi,'myi':myi}
        
    elif hem == 's':
        
        year = date.year
        ddir = sh_it_dir + f'/MultiyearIce-Antarctic-{year}vAQ2'
        
        with Dataset(f'{ddir}/MultiYearIce-Antarctic-{year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}.nc') as d:
            
            thresh = 50
            
            it = np.array(d['MYI'])
            it = 100-it
            it[it<thresh]=0
            it[it>1]=1
            fyi = it.copy()
            
            it = np.array(d['MYI'])
            it[it<thresh]=0
            it[it>1]=1
            myi = it.copy()
            
            return {'fyi':fyi,'myi':myi}

files = os.listdir(scratch)

ak_files = [f for f in files if 'ak' in f]
cs_files = [f for f in files if 'cs' in f]

ak_nh = [f for f in ak_files if 'nh' in f]
ak_nh_dates = sorted([f for f in ak_nh if 'date' in f])

cs_nh = [f for f in cs_files if 'nh' in f]
cs_nh_dates = sorted([f for f in cs_nh if 'date' in f])

ak_sh = [f for f in ak_files if 'sh' in f]
ak_sh_dates = sorted([f for f in ak_sh if 'date' in f])

cs_sh = [f for f in cs_files if 'sh' in f]
cs_sh_dates = sorted([f for f in cs_sh if 'date' in f])

lists_of_dates={'ak':{'n':ak_nh_dates,'s':ak_sh_dates},'cs':{'n':cs_nh_dates,'s':cs_sh_dates}}

def make_LOD(f,list_of_dates):
    month = int(f.split('_')[1])
    year = int(f.split('_')[2])
    
    lod = [datetime.date(year,month,x) for x in list_of_dates]
    
    return lod

if hem=='s':yearthresh=2012
elif hem =='n':yearthresh=2009

for f in tqdm.tqdm(cs_sh_dates+ak_sh_dates):
    
    data_file = f[:-8]+'.p'
    
    list_of_dates = pickle.load(open(f'{scratch}/{f}','rb'))
    
    if f[:3]=='cs2': list_of_dates = make_LOD(f,list_of_dates)
    
    list_of_data = pickle.load(open(f'{scratch}/{data_file}','rb'))
    
    dict_of_lists = {'means_fyi':[],'stds_fyi':[],'num_fyi':[],
                     'means_myi':[],'stds_myi':[],'num_myi':[],'date':[]}
    
    for date, data_on_date in list(zip(list_of_dates,list_of_data)):
        if date.year>yearthresh:
            try:

                #dictionary of fyi & myi ice types
                ice_type = get_ice_type_from_date(date,hem)

                #mask out array at low latitudes
                if hem == 's':
                    data_on_date[latgrids[hem]>-66]=np.nan
                elif hem == 'n':
                    data_on_date[latgrids[hem]<66]=np.nan

                for type_ in ['myi','fyi']:

                    # Make ice type mask by selecting from dictionary
                    ice_type_array = ice_type[type_]
                    ## Bounding box
                    
                    if bbox:
                        data_on_date[nh_longrid>-72] = np.nan
                        data_on_date[nh_longrid<-98] = np.nan
                        data_on_date[latgrids[hem]>66] = np.nan
                        data_on_date[latgrids[hem]<50] = np.nan

                    # Mask actual data with the ice type maks
                    dat = data_on_date[ice_type_array==1]
                    

        #             print(len(dat))

                    dict_of_lists[f'means_{type_}'].append(np.nanmean(dat))
                    dict_of_lists[f'stds_{type_}'].append(np.nanstd(dat))
                    dict_of_lists[f'num_{type_}'].append(len(dat[~np.isnan(dat)]))

                dict_of_lists[f'date'].append(date)

            except Exception as e:
                print(e)
            
    df = pd.DataFrame(dict_of_lists)
    
    df.to_csv('csvs/'+f[:-8]+'.csv')